# NL to FOL Translation with Llama 3.1

For this experiment, we will use the [llama 3.1 70b](https://huggingface.co/meta-llama/Meta-Llama-3.1-70B) model, hosted on [NVIDIA NIM](https://build.nvidia.com/explore/discover), to run inference on the dataset. 

## 0. Setup

In [1]:
import openai
from openai import OpenAI
from IPython.display import Markdown

from prompt import PromptTemplate, TRANSLATE_TEMPLATE_V1, TRANSLATE_EXAMPLES_V1

In [10]:
# Load environment variables
import os
from dotenv import load_dotenv
load_dotenv()

NVIDIA_API_KEY = os.environ.get("NVIDIA_API_KEY")

## 1. Test on a single instance

In [3]:
nl_str = '''
Chào PĐT, Em là Nguyễn Văn An, đang học khoa KH&KTMT, khóa 2023. Em nhận được cảnh cáo học vụ ở HK232 này, nhưng nhưng do học kỳ này em bận tham dự hoạt động tình nguyện Chiến dịch Xanh của Đoàn hội cấp tỉnh TP nên lơ là trong việc học tập (em có đính kèm minh chứng). Em muốn hỏi là liệu nhà trường có thể xem xét cho em trong trường hợp này được không? Em cảm ơn rất nhiều.
'''

existing_predicates_arr = [
    'FromFaculty(x,y)',
    'StartsYear(x,y)',
    'HasReceived(x,y)',
    'ReceivedWarningInSemester(x,y)',
    'ParticipatedIn(x,y)',
    'OrganizedBy(x,y)',
]
existing_predicates_str = '- ' + '\n- '.join(existing_predicates_arr)

In [4]:
print(nl_str)
print(existing_predicates_str)


Chào PĐT, Em là Nguyễn Văn An, đang học khoa KH&KTMT, khóa 2023. Em nhận được cảnh cáo học vụ ở HK232 này, nhưng nhưng do học kỳ này em bận tham dự hoạt động tình nguyện Chiến dịch Xanh của Đoàn hội cấp tỉnh TP nên lơ là trong việc học tập (em có đính kèm minh chứng). Em muốn hỏi là liệu nhà trường có thể xem xét cho em trong trường hợp này được không? Em cảm ơn rất nhiều.

- FromFaculty(x,y)
- StartsYear(x,y)
- HasReceived(x,y)
- ReceivedWarningInSemester(x,y)
- ParticipatedIn(x,y)
- OrganizedBy(x,y)


In [5]:
prompt_template = PromptTemplate(TRANSLATE_TEMPLATE_V1)
prompt = prompt_template(
    nl_str=nl_str,
    existing_predicates_str=existing_predicates_str,
    examples_str=TRANSLATE_EXAMPLES_V1
)

In [6]:
print(prompt)

Translate the following natural language (NL) statement to a first-order logic (FOL) rule.

When there are pre-defined predicates, try to use them to form the premises, and only add new predicates if necessary.

The output must be in JSON format and has the following 3 fields:
* `existing_predicates`: array of existing predicates that can be used to form the premises, in camel case with no space, and number of variables it takes e.g., `CamelCase(x,y)`
* `new_predicates`: array of new predicates, in camel case with no space, and number of variables it takes e.g., `CamelCase(x,y)`
* `variables`: array of variables, in lower case with no space, e.g., `lowercase`
* `premises`: array of premises constructed from the NL statement
* `conclusion`: the translation of the conclusion of the NL, can be a question or a statement. If it is a yes/no question, translate to the one with 'yes' semantic meaning.

Important notes:
* Don't forget to also include predicates used in `conclusion` in either `e

In [11]:
from openai import OpenAI

client = OpenAI(
  base_url = "https://integrate.api.nvidia.com/v1",
  api_key = NVIDIA_API_KEY
)

completion = client.chat.completions.create(
    model="meta/llama-3.1-70b-instruct",
    messages=[
        {
            "role": "user",
            "content": prompt
        }
    ],
    temperature=0.0,
    top_p=0.7,
    max_tokens=1024,
    stream=False
)

print(completion.choices[0].message.content)

```
{
  "existing_predicates": [
    "FromFaculty(x,y)",
    "StartsYear(x,y)",
    "HasReceived(x,y)",
    "ReceivedWarningInSemester(x,y)",
    "ParticipatedIn(x,y)",
    "OrganizedBy(x,y)"
  ],
  "new_predicates": [
    "IsStudent(x)",
    "IsVolunteerActivity(x)",
    "IsProvinceLevel(x)",
    "IsAttachedEvidence(x)"
  ],
  "variables": [
    "an",
    "khktmt",
    "2023",
    "hk232",
    "chienDichXanh",
    "tp"
  ],
  "premises": [
    "IsStudent(an)",
    "FromFaculty(an, khktmt)",
    "StartsYear(an, 2023)",
    "ReceivedWarningInSemester(an, hk232)",
    "ParticipatedIn(an, chienDichXanh)",
    "IsVolunteerActivity(chienDichXanh)",
    "OrganizedBy(chienDichXanh, tp)",
    "IsProvinceLevel(tp)",
    "IsAttachedEvidence(an)"
  ],
  "conclusion": "∃x (IsStudent(x) ∧ FromFaculty(x, khktmt) ∧ StartsYear(x, 2023) ∧ ReceivedWarningInSemester(x, hk232) ∧ ParticipatedIn(x, chienDichXanh) ∧ IsVolunteerActivity(chienDichXanh) ∧ OrganizedBy(chienDichXanh, tp) ∧ IsProvinceLevel(tp) ∧ I

## 2. Run on the whole dataset